### Dataframes, Spark SQL e Parquet - practica

In [1]:
# Carga o ficheiro purchases.txt desde o sistema de ficheiros localabs

In [11]:
compras = spark.read.csv('file:///home/hduser/Descargas/purchases.txt',sep='\t')
compras = compras.toDF('data', 'hora', 'tenda', 'categoria', 'vendas', 'tipo_pago')

In [2]:
# Crea unha táboa sobre o dataframe para poder realizar consultas SQL

In [12]:
compras.createOrReplaceTempView('employees')

In [3]:
# Mostra os datos do dataframe

In [13]:
compras.show()

+----------+-----+--------------+--------------------+------+----------+
|      data| hora|         tenda|           categoria|vendas| tipo_pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [4]:
# Mostra os datos da táboa cunha consulta SQL

In [14]:
spark.sql('SELECT * FROM employees').show()

+----------+-----+--------------+--------------------+------+----------+
|      data| hora|         tenda|           categoria|vendas| tipo_pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [5]:
# Conta o número de vendas por tenda

In [16]:
spark.sql("SELECT tenda, count(vendas) as vendas_tenda FROM employees GROUP BY tenda").show()

+---------------+------------+
|          tenda|vendas_tenda|
+---------------+------------+
|North Las Vegas|       40013|
|        Phoenix|       40333|
|          Omaha|       40209|
|      Anchorage|       39806|
|        Anaheim|       40086|
|     Greensboro|       40232|
|         Dallas|       40368|
|        Oakland|       39728|
|         Laredo|       40342|
|     Scottsdale|       40173|
|    San Antonio|       40197|
|    Bakersfield|       40326|
|        Raleigh|       40261|
|    Chula Vista|       40080|
|   Philadelphia|       40748|
|     Louisville|       40099|
|    Los Angeles|       40254|
|       Chandler|       39826|
|     Sacramento|       40561|
|   Indianapolis|       40321|
+---------------+------------+
only showing top 20 rows



In [6]:
# Fai un total de vendas por categoría

In [18]:
spark.sql("SELECT categoria, count(vendas) as vendas_categoria FROM employees GROUP BY categoria").show()

+--------------------+----------------+
|           categoria|vendas_categoria|
+--------------------+----------------+
| Children's Clothing|          230469|
|      Sporting Goods|          229932|
|                 CDs|          230039|
|           Computers|          229059|
|Consumer Electronics|          229761|
|   Health and Beauty|          229667|
|        Pet Supplies|          229222|
|                DVDs|          230274|
|                Baby|          230293|
|              Crafts|          229749|
|    Women's Clothing|          230050|
|         Video Games|          230237|
|               Books|          229787|
|               Music|          230150|
|      Men's Clothing|          230430|
|             Cameras|          229320|
|              Garden|          230073|
|                Toys|          229964|
+--------------------+----------------+



In [7]:
# Garda o dataframe en HDFS en formato parquet, con particións para as diferentes tendas

In [20]:
compras.write.partitionBy("tenda").mode("overwrite").parquet("employees.parquet")

In [8]:
# Carga os datos da tenda de San Jose

In [22]:
compras_parquet_partido = spark.read.parquet("employees.parquet/tenda=San Jose")

In [9]:
# Mostra os datos do dataframe

In [23]:
compras_parquet_partido.show()

+----------+-----+--------------------+------+----------+
|      data| hora|           categoria|vendas| tipo_pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [10]:
# Crea unha táboa sobre o dataframe dos datos de San Jose

In [24]:
compras_parquet_partido.createOrReplaceTempView('sanjose')

In [11]:
# Mostra os datos da táboa

In [26]:
spark.sql('SELECT * FROM sanjose').show()

+----------+-----+--------------------+------+----------+
|      data| hora|           categoria|vendas| tipo_pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [12]:
# Consulta o total de vendas de San José por tipo de elementos (categoría)

In [27]:
spark.sql("SELECT categoria, count(vendas) as vendas_categoria FROM sanjose GROUP BY categoria").show()

+--------------------+----------------+
|           categoria|vendas_categoria|
+--------------------+----------------+
| Children's Clothing|            2173|
|      Sporting Goods|            2180|
|                 CDs|            2325|
|           Computers|            2264|
|Consumer Electronics|            2240|
|   Health and Beauty|            2233|
|        Pet Supplies|            2179|
|                DVDs|            2239|
|                Baby|            2268|
|              Crafts|            2208|
|    Women's Clothing|            2199|
|         Video Games|            2298|
|               Books|            2107|
|               Music|            2220|
|      Men's Clothing|            2229|
|              Garden|            2215|
|             Cameras|            2143|
|                Toys|            2178|
+--------------------+----------------+

